In [1]:
# git clone package:
# !git clone <path_package>
# or copy package :
!cp -r /kaggle/input/autonlp-git/SentimentML-main /kaggle/working

In [2]:
import os
# directory of package :
fd = os.open("/kaggle/working/SentimentML-main", os.O_RDONLY )
os.fchdir(fd)
os.getcwd()

'/kaggle/working/SentimentML-main'

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 561 kB 527 kB/s 
     |████████████████████████████████| 7.7 MB 10.3 MB/s 
     |████████████████████████████████| 280 kB 45.9 MB/s 
     |████████████████████████████████| 11.6 MB 53.3 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 80 kB 10.1 MB/s 
     |████████████████████████████████| 23.9 MB 563 kB/s 
     |████████████████████████████████| 2.6 MB 55.0 MB/s 
     |████████████████████████████████| 283 kB 59.8 MB/s 
     |████████████████████████████████| 70 kB 6.8 MB/s 
     |████████████████████████████████| 10.8 MB 56.7 MB/s 
     |████████████████████████████████| 301 kB 56.4 MB/s 
     |████████████████████████████████| 166.7 MB 16 kB/s 
     |████████████████████████████████| 46 kB 3.1 MB/s 
     |████████████████████████████████| 12.8 MB 27.0 MB/s 
     |████████████████████████████████| 394.3 MB 11 kB/s 
     |████████████████████████████████| 47.5 MB 205 kB/s 
     |█████████████

In [4]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-2.0.0-py3-none-any.whl
     |████████████████████████████████| 2.9 MB 517 kB/s 
  Using cached huggingface_hub-0.0.17-py3-none-any.whl (52 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.3.0 requires transformers<4.1,>=4.0, but you have transformers 4.11.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [5]:
# if png doesn't save correctly
!pip install kaleido

     |████████████████████████████████| 79.9 MB 183 kB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [6]:
# if spacy can't load spacy model :
!python3 -m spacy download fr_core_news_md

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
     |████████████████████████████████| 46.1 MB 5.9 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.3.0-py3-none-any.whl size=46100669 sha256=f737d80fb7e9b49e3d3b5f85953ec22b4bede47d23b2925f5858c4a2ccd12811
  Stored in directory: /tmp/pip-ephem-wheel-cache-kt28uokr/wheels/62/a5/93/9c82d8ae84430ed80720262c11726d046d3444af301b7d2895
Successfully built fr-core-news-md
You should consider upgrading via the '/opt/conda/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


# Import

In [7]:
import pandas as pd
from autonlp.autonlp import AutoNLP
import os
from pathlib import Path
from autonlp.flags import Flags, save_yaml
import dataclasses

# Flags update (parameters)

In [8]:
#####################
# Parameters
#####################

flags_dict_info = {
    "path_data": "/kaggle/input/finance-corpus/FinancialPhraseBank_train.csv",
    "path_data_validation": "",
    "apply_logs": True,
    "outdir": "/kaggle/working/logs",
    "seed": 15,
    "debug": False,  # for debug : use only 50 data rows for training
}
flags_dict_preprocessing = {
    "column_text": "text_fr",  # name column with texts
    "target": "sentiment",     # name column with targets
    "language_text": "fr",
    "apply_small_clean": True,
    "name_spacy_model": "fr_core_news_md",
    "apply_spacy_preprocessing": True,
    "apply_entity_preprocessing": True
}

flags_dict_autonlp = {
    "objective": 'multi-class',    # 'binary' or 'multi-class' or 'regression'
    
    "embedding": {"tf": None, "tf-idf": None, "word2vec": None, "fasttext": None, "doc2vec": None, "transformer": 1},

    "clustering": {"NMF_frobenius": [], "NMF_kullback": [], "LDA": [],
                   "hdbscan": [], "ACP_hdbscan": [], "UMAP_hdbscan": [],
                   "kmeans": [], "ACP_kmeans": [], "UMAP_kmeans": [],
                   "agglomerativeclustering": [], "ACP_agglomerativeclustering": [], "UMAP_agglomerativeclustering": [],
                   "Similarity_voc": [], "zero_shot": [1]},

    "n_groups": 20,
    "average_scoring": "macro",
    
    "max_run_time_per_model": 100,
    "frac_trainset": 1,
    "scoring": 'f1',
    
    "method_embedding": {'Word2vec': 'Word2Vec',
                         'Fasttext': 'FastText',
                         'Doc2Vec': 'Doc2Vec',
                         'Transformer': 'zero-shot', # !!!
                         'spacy': [("all", False)]},
    
    "batch_size":100,
    
    "vocabulary_labels": {"positive": ["positif"],
                          "neutral": ["neutre"],
                          "negative": ["négatif"]},
    
    "pattern_zero_shot": "Le sentiment du commentaire est {}",
    
    "tr_maxlen": 100
    
}

flags_dict_display = {
    "sort_leaderboard": 'f1'
}

flags = Flags().update(flags_dict_info)
flags = flags.update(flags_dict_preprocessing)
flags = flags.update(flags_dict_autonlp)
flags = flags.update(flags_dict_display)
print("flags :", flags)
debug = flags.debug
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)
flags_dict = dataclasses.asdict(flags)
save_yaml(outdir / "flags.yaml", flags_dict)

flags : Flags(path_data='/kaggle/input/finance-corpus/FinancialPhraseBank_train.csv', path_data_validation='', apply_logs=True, outdir='/kaggle/working/logs', apply_mlflow=False, experiment_name='Experiment', apply_app=False, debug=False, seed=15, column_text='text_fr', language_text='fr', target='sentiment', apply_small_clean=True, name_spacy_model='fr_core_news_md', apply_spacy_preprocessing=True, apply_entity_preprocessing=True, objective='multi-class', embedding={'tf': None, 'tf-idf': None, 'word2vec': None, 'fasttext': None, 'doc2vec': None, 'transformer': 1}, classifier={'Naive_Bayes': [1], 'Logistic_Regression': [2], 'SGD_Classifier': [], 'XGBoost': [], 'Global_Average': [], 'Attention': [], 'BiRNN': [], 'BiRNN_Attention': [], 'biLSTM': [], 'BiLSTM_Attention': [], 'biGRU': [], 'BiGRU_Attention': []}, regressor={'SGD_Regressor': [1], 'XGBoost': [], 'Global_Average': [2], 'Attention': [], 'BiRNN': [], 'BiRNN_Attention': [], 'biLSTM': [], 'BiLSTM_Attention': [], 'biGRU': [], 'BiGRU

# Instantiation

In [9]:
autonlp = AutoNLP(flags)

# Preprocessing

In [10]:
%%time
autonlp.data_preprocessing()


Read data...

Begin preparation of 3876 data :

Training set size : 3876
Test set size : 0

Begin preprocessing of 3876 train data :
⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
- Apply small clean of texts...
- Apply nlp.pipe from spacy...
- Apply entities preprocessing...
CPU times: user 19.4 s, sys: 1.61 s, total: 21 s
Wall time: 25.2 s


# Training

In [11]:
dict_clustering_train = autonlp.fit_transform_clustering()


Fit and Transform Clustering :
List of Clustering : ['zero-shot+zero_shot']

zero-shot+zero_shot Clustering...


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 39/39 [02:25<00:00,  3.73s/it]



Scores :
accuracy = 0.3439
precision macro = 0.464
recall macro = 0.507
f1 score macro = 0.3411




# Leaderboard (Validation score)

In [12]:
leaderboard_val = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='val')
print('\nValidation Leaderboard')
leaderboard_val.to_csv(os.path.join(flags.outdir,'leaderboard_val.csv'), index=False)
leaderboard_val


Validation Leaderboard


,name,accuracy_val,recall_macro_val,precision_macro_val,f1_macro_val
0,zero-shot+zero_shot,0.3439,0.507,0.464,0.3411


# Testing on other test set

In [13]:
data_test = pd.read_csv("/kaggle/input/finance-corpus/FinancialPhraseBank_test.csv")

In [14]:
X_test, _, Y_test = autonlp.preprocess_test_data(data_test)

- Apply small clean of texts...
- Apply nlp.pipe from spacy...
- Apply entities preprocessing...


In [15]:
name_logs = 'last_logs'
on_test_data = False
dict_clustering_test = autonlp.transform_clustering(name_logs=name_logs, on_test_data=on_test_data, x = X_test, y=Y_test)


Transform Clustering :

zero-shot+zero_shot Clustering...


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 10/10 [00:38<00:00,  3.81s/it]

Distribution y_true


neutral     576
positive    275
negative    118
dtype: int64

Scores :
accuracy = 0.3664
precision macro = 0.4764
recall macro = 0.5376
f1 score macro = 0.3621




In [16]:
leaderboard_test = autonlp.get_leaderboard(sort_by=flags.sort_leaderboard, dataset='test') #, info_models=autonlp.info_models)
print('\nTest Leaderboard')
leaderboard_test


Test Leaderboard


,name,accuracy_test,recall_macro_test,precision_macro_test,f1_macro_test
0,zero-shot+zero_shot,0.3664,0.5376,0.4764,0.3621
